# TP : Réseaux de Neurones Récurrents (RNN)

## Objectifs
- Comprendre les concepts fondamentaux des RNN (Simple RNN, LSTM, GRU)
- Implémenter des modèles RNN pour l'analyse de sentiment
- Comparer les performances des différents types de RNN
- Visualiser le processus d'apprentissage
- Comprendre les applications pratiques des RNN


## 1. Imports et Configuration

Importation des bibliothèques nécessaires pour les RNN avec Keras/TensorFlow.


In [3]:
!pip install tensorflow numpy pandas seaborn scikit-learn


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Data manipulation
import numpy as np
import pandas as pd
import os
import re

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Configuration
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

# Style pour les graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Vérifier la disponibilité du GPU
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

# Paramètres globaux
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)


## 2. Introduction aux RNN

### 2.1. Concepts Fondamentaux

Les **Réseaux de Neurones Récurrents (RNN)** sont conçus pour traiter des données séquentielles (texte, séries temporelles, audio, etc.).

**Caractéristiques principales :**
- **Mémoire** : Les RNN maintiennent une mémoire des informations précédentes dans la séquence
- **Paramètres partagés** : Les mêmes poids sont utilisés à chaque pas de temps
- **Propagation dans le temps** : L'information circule à travers les pas de temps

**Types de RNN :**
1. **Simple RNN** : Architecture de base, mais souffre du problème de gradient qui disparaît
2. **LSTM (Long Short-Term Memory)** : Résout le problème du gradient qui disparaît avec des portes (forget, input, output)
3. **GRU (Gated Recurrent Unit)** : Variante simplifiée de LSTM avec moins de paramètres


### 2.2. Application Pratique : Analyse de Sentiment

Dans ce TP, nous allons utiliser les RNN pour classer le sentiment de critiques de films comme **positif** ou **négatif**.

**Dataset** : Nous utiliserons le dataset IMDb (Internet Movie Database) disponible dans Keras.


## 3. Chargement et Préparation des Données

### 3.1. Chargement du Dataset IMDb


In [ ]:
# Chargement du dataset IMDb
print("Chargement du dataset IMDb...")
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

print(f"Nombre d'échantillons d'entraînement: {len(x_train)}")
print(f"Nombre d'échantillons de test: {len(x_test)}")
print(f"\nExemple de séquence (premiers 20 indices): {x_train[0][:20]}")
print(f"Label (0 = négatif, 1 = positif): {y_train[0]}")

# Statistiques sur les séquences
train_lengths = [len(seq) for seq in x_train]
test_lengths = [len(seq) for seq in x_test]

print(f"\nLongueur moyenne des séquences d'entraînement: {np.mean(train_lengths):.2f}")
print(f"Longueur max des séquences d'entraînement: {max(train_lengths)}")
print(f"Longueur min des séquences d'entraînement: {min(train_lengths)}")


### 3.2. Décodage des Textes (optionnel)

Décodons quelques exemples pour voir les textes originaux.


In [ ]:
# Charger le dictionnaire des mots (index -> mot)
word_index = keras.datasets.imdb.get_word_index()

# Créer un dictionnaire inverse (index -> mot)
reverse_word_index = {value: key for (key, value) in word_index.items()}
reverse_word_index[0] = "<PAD>"  # Padding
reverse_word_index[1] = "<START>"  # Start of sequence
reverse_word_index[2] = "<UNK>"  # Unknown word

def decode_review(encoded_review):
    """Décode une séquence d'indices en texte"""
    return ' '.join([reverse_word_index.get(i, '?') for i in encoded_review])

# Afficher quelques exemples
print("Exemple 1 (Label: {}):".format("Positif" if y_train[0] == 1 else "Négatif"))
print(decode_review(x_train[0][:50]))  # Premiers 50 mots
print("\n" + "="*80 + "\n")

print("Exemple 2 (Label: {}):".format("Positif" if y_train[1] == 1 else "Négatif"))
print(decode_review(x_train[1][:50]))
print("\n" + "="*80 + "\n")

print("Exemple 3 (Label: {}):".format("Positif" if y_train[2] == 1 else "Négatif"))
print(decode_review(x_train[2][:50]))


### 3.3. Padding et Truncation des Séquences

Les RNN nécessitent des séquences de longueur fixe. Nous allons :
- **Padding** : Ajouter des zéros aux séquences courtes
- **Truncation** : Tronquer les séquences longues


In [ ]:
# Paramètres de preprocessing
MAX_LENGTH = 500  # Longueur maximale des séquences
VOCAB_SIZE = 10000  # Taille du vocabulaire

# Padding et truncation
x_train_padded = pad_sequences(x_train, maxlen=MAX_LENGTH, padding='pre', truncating='pre')
x_test_padded = pad_sequences(x_test, maxlen=MAX_LENGTH, padding='pre', truncating='pre')

print(f"Shape des données d'entraînement après padding: {x_train_padded.shape}")
print(f"Shape des données de test après padding: {x_test_padded.shape}")
print(f"\nExemple de séquence après padding (premiers 30 éléments):")
print(x_train_padded[0][:30])


### 3.4. Visualisation des Données

Visualisons la distribution des longueurs de séquences et des labels.


In [ ]:
# Visualisation de la distribution des longueurs
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Distribution des longueurs
axes[0].hist(train_lengths, bins=50, alpha=0.7, label='Train', color='skyblue')
axes[0].hist(test_lengths, bins=50, alpha=0.7, label='Test', color='lightcoral')
axes[0].axvline(MAX_LENGTH, color='red', linestyle='--', linewidth=2, label=f'Max Length ({MAX_LENGTH})')
axes[0].set_xlabel('Longueur de la séquence')
axes[0].set_ylabel('Fréquence')
axes[0].set_title('Distribution des Longueurs de Séquences', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Distribution des labels
train_labels_count = pd.Series(y_train).value_counts()
test_labels_count = pd.Series(y_test).value_counts()

x_pos = np.arange(2)
width = 0.35

axes[1].bar(x_pos - width/2, [train_labels_count[0], train_labels_count[1]], 
           width, label='Train', color='skyblue', alpha=0.7)
axes[1].bar(x_pos + width/2, [test_labels_count[0], test_labels_count[1]], 
           width, label='Test', color='lightcoral', alpha=0.7)
axes[1].set_xlabel('Label')
axes[1].set_ylabel('Nombre d\'échantillons')
axes[1].set_title('Distribution des Labels', fontsize=14, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(['Négatif (0)', 'Positif (1)'])
axes[1].legend()
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nDistribution des labels d'entraînement:")
print(f"Négatif (0): {train_labels_count[0]} ({train_labels_count[0]/len(y_train)*100:.1f}%)")
print(f"Positif (1): {train_labels_count[1]} ({train_labels_count[1]/len(y_train)*100:.1f}%)")


## 4. Construction des Modèles RNN

Nous allons créer et comparer trois types de modèles RNN :
1. **Simple RNN**
2. **LSTM (Long Short-Term Memory)**
3. **GRU (Gated Recurrent Unit)**


### 4.1. Modèle Simple RNN


In [ ]:
def create_simple_rnn(embedding_dim=128, rnn_units=64):
    """
    Crée un modèle Simple RNN pour l'analyse de sentiment.
    
    Architecture:
    - Embedding: Convertit les indices en vecteurs denses
    - Simple RNN: Traite la séquence
    - Dense: Classification finale
    """
    model = models.Sequential([
        layers.Embedding(VOCAB_SIZE, embedding_dim, input_length=MAX_LENGTH),
        layers.SimpleRNN(rnn_units, dropout=0.2, recurrent_dropout=0.2),
        layers.Dense(1, activation='sigmoid')  # Classification binaire
    ])
    
    return model

# Créer le modèle
model_simple_rnn = create_simple_rnn()

# Compiler le modèle
model_simple_rnn.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Afficher l'architecture
print("Architecture du modèle Simple RNN:")
model_simple_rnn.summary()


### 4.2. Modèle LSTM (Long Short-Term Memory)


In [ ]:
def create_lstm_model(embedding_dim=128, lstm_units=64):
    """
    Crée un modèle LSTM pour l'analyse de sentiment.
    
    Architecture:
    - Embedding: Convertit les indices en vecteurs denses
    - LSTM: Traite la séquence avec mémoire à long terme
    - Dense: Classification finale
    """
    model = models.Sequential([
        layers.Embedding(VOCAB_SIZE, embedding_dim, input_length=MAX_LENGTH),
        layers.LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2),
        layers.Dense(1, activation='sigmoid')  # Classification binaire
    ])
    
    return model

# Créer le modèle
model_lstm = create_lstm_model()

# Compiler le modèle
model_lstm.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Afficher l'architecture
print("Architecture du modèle LSTM:")
model_lstm.summary()


### 4.3. Modèle GRU (Gated Recurrent Unit)


In [ ]:
def create_gru_model(embedding_dim=128, gru_units=64):
    """
    Crée un modèle GRU pour l'analyse de sentiment.
    
    Architecture:
    - Embedding: Convertit les indices en vecteurs denses
    - GRU: Traite la séquence avec portes (simplification de LSTM)
    - Dense: Classification finale
    """
    model = models.Sequential([
        layers.Embedding(VOCAB_SIZE, embedding_dim, input_length=MAX_LENGTH),
        layers.GRU(gru_units, dropout=0.2, recurrent_dropout=0.2),
        layers.Dense(1, activation='sigmoid')  # Classification binaire
    ])
    
    return model

# Créer le modèle
model_gru = create_gru_model()

# Compiler le modèle
model_gru.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Afficher l'architecture
print("Architecture du modèle GRU:")
model_gru.summary()


### 4.4. Modèle LSTM Bidirectionnel (Optionnel - Performance Améliorée)

Les LSTM bidirectionnels traitent la séquence dans les deux sens (avant et arrière), ce qui peut améliorer les performances.


In [ ]:
def create_bidirectional_lstm(embedding_dim=128, lstm_units=64):
    """
    Crée un modèle LSTM bidirectionnel pour l'analyse de sentiment.
    
    Architecture:
    - Embedding: Convertit les indices en vecteurs denses
    - Bidirectional LSTM: Traite la séquence dans les deux sens
    - Dense: Classification finale
    """
    model = models.Sequential([
        layers.Embedding(VOCAB_SIZE, embedding_dim, input_length=MAX_LENGTH),
        layers.Bidirectional(layers.LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2)),
        layers.Dense(1, activation='sigmoid')  # Classification binaire
    ])
    
    return model

# Créer le modèle
model_bidirectional_lstm = create_bidirectional_lstm()

# Compiler le modèle
model_bidirectional_lstm.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Afficher l'architecture
print("Architecture du modèle LSTM Bidirectionnel:")
model_bidirectional_lstm.summary()


## 5. Entraînement des Modèles

Nous allons entraîner chaque modèle et comparer leurs performances.


In [ ]:
# Configuration de l'entraînement
EPOCHS = 10
BATCH_SIZE = 64
VALIDATION_SPLIT = 0.2

# Callbacks pour améliorer l'entraînement
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=0.00001,
        verbose=1
    )
]

print("Configuration de l'entraînement:")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Validation split: {VALIDATION_SPLIT}")


In [ ]:
# Dictionnaire pour stocker les historiques
histories = {}

# Liste des modèles à entraîner
models_to_train = {
    'Simple RNN': model_simple_rnn,
    'LSTM': model_lstm,
    'GRU': model_gru,
    'Bidirectional LSTM': model_bidirectional_lstm
}


In [ ]:
# Entraînement de chaque modèle
for model_name, model in models_to_train.items():
    print(f"\n{'='*70}")
    print(f"Entraînement du modèle: {model_name}")
    print(f"{'='*70}\n")
    
    # Entraîner le modèle
    history = model.fit(
        x_train_padded, y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_split=VALIDATION_SPLIT,
        callbacks=callbacks,
        verbose=1
    )
    
    # Stocker l'historique
    histories[model_name] = history
    
    # Évaluation sur le test set
    test_loss, test_accuracy = model.evaluate(x_test_padded, y_test, verbose=0)
    print(f"\nRésultats sur le test set:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")


## 6. Visualisation des Résultats d'Entraînement

Comparons les courbes d'apprentissage de chaque modèle.


In [ ]:
def plot_training_history(histories_dict):
    """Visualise l'historique d'entraînement pour tous les modèles"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Loss - Training
    axes[0, 0].set_title('Training Loss', fontsize=14, fontweight='bold')
    for model_name, history in histories_dict.items():
        axes[0, 0].plot(history.history['loss'], label=model_name, linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # Loss - Validation
    axes[0, 1].set_title('Validation Loss', fontsize=14, fontweight='bold')
    for model_name, history in histories_dict.items():
        axes[0, 1].plot(history.history['val_loss'], label=model_name, linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    # Accuracy - Training
    axes[1, 0].set_title('Training Accuracy', fontsize=14, fontweight='bold')
    for model_name, history in histories_dict.items():
        axes[1, 0].plot(history.history['accuracy'], label=model_name, linewidth=2)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    # Accuracy - Validation
    axes[1, 1].set_title('Validation Accuracy', fontsize=14, fontweight='bold')
    for model_name, history in histories_dict.items():
        axes[1, 1].plot(history.history['val_accuracy'], label=model_name, linewidth=2)
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Accuracy')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Visualiser les résultats
plot_training_history(histories)


## 7. Comparaison des Performances

Comparons les performances finales de tous les modèles.


In [ ]:
# Évaluation de tous les modèles sur le test set
results = {}

for model_name, model in models_to_train.items():
    test_loss, test_accuracy = model.evaluate(x_test_padded, y_test, verbose=0)
    
    # Prédictions
    y_pred_probs = model.predict(x_test_padded, verbose=0)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()
    
    # Métriques supplémentaires
    from sklearn.metrics import precision_score, recall_score, f1_score
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    results[model_name] = {
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }

# Créer un DataFrame pour la comparaison
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('test_accuracy', ascending=False)

print("\n" + "="*70)
print("COMPARAISON DES MODÈLES")
print("="*70)
print(results_df.to_string())

# Visualisation graphique
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

metrics = ['test_accuracy', 'precision', 'recall', 'f1_score']
titles = ['Test Accuracy', 'Precision', 'Recall', 'F1-Score']

for idx, (metric, title) in enumerate(zip(metrics, titles)):
    row = idx // 2
    col = idx % 2
    
    axes[row, col].bar(results_df.index, results_df[metric], color='steelblue', alpha=0.7)
    axes[row, col].set_title(title, fontsize=14, fontweight='bold')
    axes[row, col].set_ylabel(metric.replace('_', ' ').title())
    axes[row, col].tick_params(axis='x', rotation=45)
    axes[row, col].grid(axis='y', alpha=0.3)
    axes[row, col].set_ylim([0, 1])

plt.tight_layout()
plt.show()


### 7.1. Matrices de Confusion

Visualisons les matrices de confusion pour chaque modèle.


In [ ]:
# Calcul du nombre de lignes et colonnes pour la grille
n_models = len(models_to_train)
n_cols = 2
n_rows = (n_models + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 6*n_rows))
axes = axes.flatten() if n_models > 1 else [axes]

for idx, (model_name, model) in enumerate(models_to_train.items()):
    # Prédictions
    y_pred_probs = model.predict(x_test_padded, verbose=0)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(
        cm, 
        annot=True, 
        fmt='d', 
        cmap='Blues',
        ax=axes[idx],
        xticklabels=['Négatif', 'Positif'],
        yticklabels=['Négatif', 'Positif']
    )
    
    accuracy = results[model_name]['test_accuracy']
    axes[idx].set_title(f'{model_name}\nAccuracy: {accuracy:.4f}', 
                       fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Vraie classe')
    axes[idx].set_xlabel('Classe prédite')

# Masquer les axes inutilisés
for idx in range(n_models, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()


### 7.2. Rapports de Classification Détaillés

Affichons les rapports de classification pour le meilleur modèle.


In [ ]:
# Trouver le meilleur modèle
best_model_name = results_df.index[0]
best_model = models_to_train[best_model_name]

print(f"\n{'='*70}")
print(f"Rapport de Classification - {best_model_name}")
print(f"{'='*70}\n")

# Prédictions du meilleur modèle
y_pred_probs = best_model.predict(x_test_padded, verbose=0)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Rapport de classification
print(classification_report(y_test, y_pred, target_names=['Négatif', 'Positif']))


## 8. Prédictions sur Nouvelles Phrases

Testons le modèle sur de nouvelles critiques que nous créons.


In [ ]:
def predict_sentiment(model, text, word_index_dict, max_length=MAX_LENGTH):
    """
    Prédit le sentiment d'une nouvelle phrase.
    
    Args:
        model: Modèle entraîné
        text: Texte à analyser
        word_index_dict: Dictionnaire word_index
        max_length: Longueur maximale de la séquence
    
    Returns:
        sentiment: 'Positif' ou 'Négatif'
        confidence: Probabilité de confiance
    """
    # Nettoyer le texte
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    
    # Convertir en indices
    sequence = [word_index_dict.get(word, 2) for word in words]  # 2 = <UNK>
    
    # Padding
    sequence = pad_sequences([sequence], maxlen=max_length, padding='pre', truncating='pre')
    
    # Prédiction
    prediction = model.predict(sequence, verbose=0)[0][0]
    sentiment = 'Positif' if prediction > 0.5 else 'Négatif'
    confidence = prediction if prediction > 0.5 else 1 - prediction
    
    return sentiment, confidence, prediction

# Exemples de nouvelles critiques
new_reviews = [
    "This movie was absolutely fantastic! The acting was superb and the plot was engaging.",
    "I hated this film. It was boring and poorly acted. I want my money back!",
    "The movie was okay. Nothing special, but not terrible either.",
    "Brilliant cinematography and excellent performances from all actors. Highly recommend!",
    "This is the worst movie I have ever seen. Terrible script and bad direction.",
    "A masterpiece of cinema. The director's vision is remarkable and the story is compelling.",
    "Not bad, but could have been better. Some good moments but overall disappointing.",
    "One of the best films of the year! Outstanding storytelling and character development."
]

print("Prédictions sur de nouvelles critiques:\n")
print("="*80)

for i, review in enumerate(new_reviews, 1):
    sentiment, confidence, prob = predict_sentiment(best_model, review, word_index)
    print(f"\nCritique {i}:")
    print(f"Texte: {review[:80]}...")
    print(f"Sentiment prédit: {sentiment} (Confiance: {confidence:.2%}, Probabilité: {prob:.4f})")
    print("-" * 80)


## 9. Analyse Comparative des Architectures RNN

Comparons visuellement les performances des différents types de RNN.


In [ ]:
# Créer un graphique comparatif
fig, ax = plt.subplots(figsize=(12, 8))

models_names = list(results_df.index)
accuracies = results_df['test_accuracy'].values
colors = ['skyblue', 'lightcoral', 'lightgreen', 'plum']

bars = ax.barh(models_names, accuracies, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)

# Ajouter les valeurs sur les barres
for i, (bar, acc) in enumerate(zip(bars, accuracies)):
    width = bar.get_width()
    ax.text(width + 0.005, bar.get_y() + bar.get_height()/2, 
            f'{acc:.4f} ({acc*100:.2f}%)', 
            ha='left', va='center', fontweight='bold', fontsize=11)

ax.set_xlabel('Test Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Comparaison des Performances des Modèles RNN', 
            fontsize=16, fontweight='bold', pad=20)
ax.set_xlim([0, 1.05])
ax.grid(axis='x', alpha=0.3, linestyle='--')
ax.set_axisbelow(True)

plt.tight_layout()
plt.show()

# Résumé textuel
print("\n" + "="*70)
print("RÉSUMÉ COMPARATIF")
print("="*70)
print(f"\nMeilleur modèle: {best_model_name}")
print(f"Accuracy: {results[best_model_name]['test_accuracy']:.4f} ({results[best_model_name]['test_accuracy']*100:.2f}%)")
print(f"\nClassement des modèles:")
for i, (model_name, metrics) in enumerate(results.items(), 1):
    print(f"{i}. {model_name}: {metrics['test_accuracy']:.4f} ({metrics['test_accuracy']*100:.2f}%)")


## 10. Sauvegarde du Meilleur Modèle

Sauvegardons le meilleur modèle pour une utilisation ultérieure.


In [ ]:
# Sauvegarder le meilleur modèle
model_filename = f'best_rnn_model_{best_model_name.lower().replace(" ", "_")}.h5'
best_model.save(model_filename)
print(f"Modèle sauvegardé: {model_filename}")

# Sauvegarder aussi les poids
weights_filename = f'best_rnn_weights_{best_model_name.lower().replace(" ", "_")}.h5'
best_model.save_weights(weights_filename)
print(f"Poids sauvegardés: {weights_filename}")

print("\nPour charger le modèle plus tard:")
print(f"loaded_model = keras.models.load_model('{model_filename}')")


## 11. Conclusions et Explications

### 11.1. Différences entre les Types de RNN

**Simple RNN :**
- Architecture la plus basique
- Souffre du problème du gradient qui disparaît (vanishing gradient)
- Limité pour les séquences longues
- Plus rapide mais moins performant

**LSTM (Long Short-Term Memory) :**
- Résout le problème du gradient qui disparaît
- Utilise des portes (forget, input, output) pour contrôler l'information
- Peut apprendre des dépendances à long terme
- Plus de paramètres, donc plus lent mais plus performant

**GRU (Gated Recurrent Unit) :**
- Compromis entre Simple RNN et LSTM
- Moins de paramètres que LSTM (pas de porte output séparée)
- Généralement aussi performant que LSTM mais plus rapide
- Bon choix pour de nombreux cas d'usage

**Bidirectional LSTM :**
- Traite la séquence dans les deux sens (avant et arrière)
- Capture le contexte complet de la séquence
- Généralement le plus performant mais aussi le plus coûteux en calcul


### 11.2. Applications des RNN

Les RNN sont utilisés dans de nombreux domaines :

1. **Traitement du Langage Naturel (NLP)** :
   - Analyse de sentiment
   - Traduction automatique
   - Génération de texte
   - Chatbots

2. **Séries Temporelles** :
   - Prédiction de cours boursiers
   - Prévisions météorologiques
   - Analyse de données sensorielles

3. **Reconnaissance Vocale** :
   - Transcription audio
   - Commandes vocales
   - Reconnaissance du locuteur

4. **Génération de Contenu** :
   - Génération de musique
   - Génération de texte créatif
   - Résumé automatique
